### Установки

In [14]:
!pip install pymorphy2
%pip install spacy
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-23 08:03:20.466724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 08:03:21.934513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


### Импорт

In [15]:
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from random import randint
from pprint import pprint

import pandas as pd
import numpy as np
import pickle
import nltk
import json
import gc
import re



*   распаковка json
*   знакомство с данными
*   разбиение текстов на  предложения
*   лемматизация и очищение от стоп-слов



### распаковка json, приведение к удобному виду для pandas

In [ ]:
with open('/content/train.json', 'r', encoding='utf-8') as f:
  json_data = json.load(f)
len(json_data)

FileNotFoundError: ignored

In [ ]:
json_data[0]['extracted_part']

In [ ]:
json_data[0]['text'][1279:1343]


In [ ]:
lenght = len(json_data)
for i in range(lenght):
  target = json_data[i]['extracted_part']['text'][0]
  if target:
    json_data[i]['target'] = target

  else: 
    json_data[i]['target'] = np.nan

  del json_data[i]['extracted_part']
  
json_data[0]

In [ ]:
with open('train_unpacked.json', 'w') as f:
  json.dump(json_data, f)

In [ ]:
del json_data
gc.collect()

### откроем в pandas посмотрим на данные

In [ ]:
df = pd.read_json('/content/train_unpacked.json', encoding='utf-8')
df.sample(3)

,id,text,label,target
164,621939083,13 ПРОЕКТ КОНТРАКТ №____ на поставку товаров г...,обеспечение исполнения контракта,Размер обеспечения исполнения Контракта состав...
217,56810761,32 УТВЕРЖДАЮ Начальник департамента городского...,обеспечение гарантийных обязательств,Размер обеспечения гарантийных обязательств по...
183,981282799,Извещение о проведении запроса котировок в эле...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 24578....


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1799 non-null   int64 
 1   text    1799 non-null   object
 2   label   1799 non-null   object
 3   target  1492 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.3+ KB


Посмотрим поближе на сами тексты

In [ ]:
df['label'].value_counts()

обеспечение исполнения контракта        988
обеспечение гарантийных обязательств    811
Name: label, dtype: int64

In [ ]:
string = randint(0, 1799)
df.loc[string, 'text'], df.loc[string, 'label'], df.loc[string, 'target'] 

('Утверждена: Начальник МКУ «ХЭГ» ______________ Шигорин С. В наименование и описание объекта закупки, условия контракта (РАЗДЕЛ 2 ИЗВЕЩЕНИЯ ОБ ОСУЩЕСТВЛЕНИИ ЗАКУПКИ) 1 Информация о заказчике Бюджетополучатель: Муниципальное казенное учреждение «Хозяйственно-эксплуатационная ыть только субъекты малого предпринимательства, социально ориентированные некоммерческие организации. 14 Требование, установленное в соответствии с частью 5 статьи 30 Федерального закона от 05.04.2013 №44-ФЗ, с указанием в соответствии с частью 6 статьи 30 Федерального закона от 05.04.2013 № 44-ФЗ объема привлечения к исполнению контрактов субподрядчиков, соисполнителей из числа субъектов малого предпринимательства, социально ориентированных некоммерческих организаций Не установлено 15 Условия, запреты и ограничения допуска товаров, происходящих из иностранного государства или группы иностранных государств (в соответствии со ст. 14 Федерального закона от 05.04.2013 № 44-ФЗ) Не установлено 16 Установить обеспечение 

В текстах не везде расставлены знаки препинания, поэтому мы не можем чётко разделить предложения и можем ориентироваться только на регистр

### Разбиение текста на предложения 

Цель - добиться, чтобы в каждом разбиении присутствовал фрагмент соответствующий таргету. Это поможет сильно улучшить accuracy

In [16]:
df = pd.read_json('/content/train_unpacked.json', encoding='utf-8')

In [ ]:
random_string = randint(0, 1799)
df.loc[random_string, :].values

array([432042942,
       'Приложение к распоряжению администрации МО «Харабалинский район» от 13.09.2022 № 735-р ИЗВЕЩЕНИЕ ОБ ЭЛЕКТРОННОМ АУКЦИОНЕ на поставку ограждения для общественного кладбища (в новой редакции) Извещение о проведении электронного аукциона Общая  в том числе с учетом положений статьи 37 настоящего Федерального закона, в случае предоставления таким участником закупки информации, содержащейся в реестре контрактов, заключенных заказчиками, и подтверждающей исполнение таким участником (без учета правопреемства) в течение трех лет до даты подачи заявки на участие в закупке трех контрактов, исполненных без применения к такому участнику неустоек (штрафов, пеней). Такая информация предоставляется участником закупки до заключения контракта в случаях, установленных настоящим Федеральным законом для предоставления обеспечения исполнения контракта. При этом сумма цен таких контрактов должна составлять не менее (начальной) максимальной цены контракта, указанного в извещении об 

Зададим несколько правил для разбиения:

*   Если в предложении на конце стоит знак, его нужно сохранить

*   Предложение не должно оканчиваться на цифры с точками (3.4.)

*   Если встречаются слова с большой буквы в предложении - они  должны остаться с большой буквы

*   Если фрагмент оканчивается на , или - тогда в конце знак не ставится


*   Чтобы отследить фрагменты, которые не разделены знаками препинания, или заглавными буквами, можно составить список слов, на которых скорее всего оканчивается искомое предложение



Таким образом, придётся расширить исходные тексты и хранить ещё и фрагменты разделений по знакам , и -

In [43]:
def is_in_stop_list(word):
  stop_list = ['заказчик', 'получател', 'договор', 'поставщик', 
      'федер', 'закон', 'росси', 'гражданск', 
      'министерств', 'директор', 'сторон', 'оборудован', 
      'акт', 'мвд', 'нмцк', 'удмур', 'глав', 'москв', 'контракт', 
      'исполнител', 'ндс', 'правительств', 'рф', 'документ', 
      'продукци', 'проект', 'цены']

  for exception in stop_list:
    word = word.lower()
    if word.startswith(exception):
      return True
  return False

In [46]:
def is_common_target_end(sentence):
  # в стоп листе сохраним падежи, чтобы не генерить много фрагментов
  stop_list = [
      'контракта', 'нцмд', 'копейки', 'договора', 'рублей', '(цены лота)', 'ндс'
  ]

  index_list = []

  word_list = sentence.split()
  for word in word_list:
    if word.lower() in stop_list:
      index_list.extend([m.start() + len(word) for m in re.finditer(word,  sentence)])


  return list(set(index_list))

In [30]:
def comma_separate(list_of_sentences, min_sentence_lenght=15):
  result_fragments_list = []

  def separate(sentence):
    list_of_commas_index = []
    list_of_comma_separated_fragments = []
    for i in range(len(sentence)):
      try:
        if sentence[i] == ',' or sentence[i] == '-':
          if not sentence[i-1].isnumeric() and not sentence[i-1].isupper() and not sentence[i+1].isnumeric() and not sentence[i+1].isupper():
            list_of_commas_index.append(i)
      except:
        pass

    list_of_commas_index.extend(is_common_target_end(sentence))
    start = 0
    for end in list_of_commas_index:
      fragment = sentence[start:end]
      list_of_comma_separated_fragments.append(fragment)
    
    return list(set(list_of_comma_separated_fragments))

  
  for sentence in list_of_sentences:
    if len(sentence) > min_sentence_lenght:
      result_fragments_list.extend(separate(sentence))

  return result_fragments_list

In [31]:
def check_the_last_word(list_of_words):
  try: 
    if int(list_of_words[-1]) < 100:
      return list_of_words[:-1]
  except:
    if '<' in list_of_words[-1] and '>' in list_of_words[-1]:
      return list_of_words[:-1]
  return list_of_words

In [32]:
def splitter(text, min_sentence_lenght=6):
  list_of_sentences = []
  list_of_words = text.split()
  sentence = []
  previous_word = ''
  for word in list_of_words:
    try:
      if not len(word) == 1:
        # критерии начала следующего предложения
        if word[-1] == '.' and word[-2].isnumeric():
          sentence = check_the_last_word(sentence)
          list_of_sentences.append(' '.join(sentence))
          sentence = []
        if word[0].isupper() and not word.isupper():
          if not is_in_stop_list(word) and sentence:
            sentence = check_the_last_word(sentence)
            list_of_sentences.append(' '.join(sentence))
            sentence = []
        if is_in_stop_list(word) and previous_word[-1] == '.':
          sentence = check_the_last_word(sentence)
          list_of_sentences.append(' '.join(sentence))
          sentence = []

      else:
        if word.isupper():
          sentence = check_the_last_word(sentence)
          list_of_sentences.append(' '.join(sentence))
          sentence = []

    except:
      pass
    sentence.append(word)

    if word is list_of_words[-1]:
      sentence = check_the_last_word(sentence)
      list_of_sentences.append(' '.join(sentence))
    previous_word = word

  # добавим фрагменты к списку предложений если extended=True
  # if extended:
  list_of_sentences.extend(comma_separate(list_of_sentences))

  for sentence in list_of_sentences:
    if len(sentence) < min_sentence_lenght or not sentence:
      del list_of_sentences[list_of_sentences.index(sentence)]
  return list_of_sentences
      

In [33]:
random_string = randint(0, 1799)
sample_text = df.loc[random_string, 'text']

In [23]:
splitted = splitter(sample_text)
for i, value in enumerate(splitted):
  if value:
    print(f'{i+1}) {value}')

print(df.loc[random_string, 'target'], df.loc[random_string, 'target'] in splitted)

1) ДОГОВОР № __________ подряда на выполнение строительно-монтажных работ с предоставлением заказчиком материалов
2) Московская область «___» _________ 20__ г.
3) Акционерное
4) Общество «Мособлгаз» (АО «Мособлгаз»), именуемое в дальнейшем «Заказчик», в лице ____________________________________, Договора на тех же условиях и в том же размере, которые указаны в настоящем разделе Договора.
5) Действие указанного пункта не распространяется на случаи, когда
6) Подрядчиком предоставлена независимая гарантия, не соответствующая требованиям законодательства Российской Федерации.
7) В ходе исполнения Договора
8) Подрядчик вправе предоставить Заказчику обеспечение исполнения Договора, уменьшенное на размер исполненных обязательств, предусмотренных Договором, взамен ранее предоставленного обеспечения исполнения Договора.
9) При этом может быть изменен способ обеспечения исполнения Договора.
10) Обеспечение исполнения Договора возвращается
11) Подрядчику при условии надлежащего исполнения
12) Под

Помотрим, как хорошо мы разделили предложения: для этого напишем функцию, которая будет считать процент вхождения таргета в разбиение. 

In [39]:
def division_check(df:pd.DataFrame):
  counter = 0
  try:
    df.drop(columns=['not_found'], inplace=True)
  except:
    pass

  for i in range(len(df)):
    try:
      if df.loc[i, 'target'] in splitter(df.loc[i, 'text']):
        df.loc[i, 'not_found'] = 1
        counter += 1
      else:
        df.loc[i, 'not_found'] = 0
    except IndexError:
      print(df.loc[i, 'text'])
      print(i)
  return counter / 1492, df[df['not_found']==0].reset_index(drop=True), df[df['not_found']==1].reset_index(drop=True)  # 1492 - ненулевые значения в таргете

In [40]:
def split_df_texts_by_sentences(data_frame:pd.DataFrame):
  for i in range(len(data_frame)-1):
    splitted = splitter(data_frame.loc[i, 'text'])
    data_frame.at[i, 'text'] = splitted
  return data_frame


In [47]:
precision, not_found, found = division_check(df)

In [48]:
precision

0.732573726541555

после исправления функции is_common_target_end удалось поднять precision с 0.63 до 0.73 и accuracy у базовой модели с 0.64 до 

In [ ]:
df_splitted = split_df_texts_by_sentences(df)

In [ ]:
df_splitted.to_json('train_with_splitted_texts_extended.json')

### Сохранение доп. датасетов для классификации.

In [ ]:
contract_enforcement = df[df['label'] == 'обеспечение исполнения контракта']['target'].values
garantee_enforcement = df[df['label'] == 'обеспечение гарантийных обязательств']['target'].values

In [ ]:
found = split_df_texts_by_sentences(found)

In [ ]:
# nonetype_sentences = df[df['target'].isna()]['text'].values
for i in range(len(found)):
  splitted_sentences = []
  for k in range(len(found.loc[i, 'text'])):
    if not found.loc[i, 'text'][k].startswith(found.loc[i, 'target']):
      splitted_sentences.append(found.loc[i, 'text'][k])

  found.at[i, 'text'] = splitted_sentences


In [ ]:
found.head()

,id,text,label,target,not_found
0,809436509,[Извещение о проведении открытого конкурса в э...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,1.0
1,854885310,"[ТРЕБОВАНИЯ, К СОДЕРЖАНИЮ ЗАЯВКИ участника зап...",обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,1.0
2,4382157,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1.0
3,184555082,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1.0
4,211645258,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,1.0


In [ ]:
nonetype = found.loc[:, 'text'].values

In [ ]:
nonetype_sentences = []

for i in range(len(nonetype)):
  for j in range(len(nonetype[i])):
    if len(nonetype[i][j]) > 5:
      nonetype_sentences.append(nonetype[i][j])

nonetype_sentences = np.array(nonetype_sentences)
len(nonetype_sentences)

46897

In [ ]:
with open('nonetype.pkl', 'wb') as fp:
  pickle.dump(nonetype_sentences, fp)

также сохраним таргеты двух классов для модели classy classification

In [ ]:
with open('contract_enforcement.pkl', 'wb') as fp:
  pickle.dump(contract_enforcement, fp)

### Приведение к нормальной форме всех датасетов


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatization(text, stopwords, morph):
  if text and len(text) > 1:
    pattern = "[A-Za-z0-9!#$%&'()*+№,./:;<=>?@[\]^_`{|}~—\"\-]+"
    text = re.sub(pattern, ' ', text)
    tokens = []
    for token in text.split():
        if token and token not in stopwords:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    return tokens
  else: 
    return None

In [ ]:
%%time
for i in range(len(df_splitted)):
  lemmatized_text = []

  for k in range(len(df_splitted.loc[i, 'text'])):
    sentence = df_splitted.loc[i, 'text'][k]
    sentence =  lemmatization(sentence, stopwords_ru, morph)
    lemmatized_text.append(sentence)
  df_splitted.at[i, 'text'] = lemmatized_text

CPU times: user 3min 12s, sys: 595 ms, total: 3min 12s
Wall time: 3min 16s


In [ ]:
df_splitted.loc[2, 'text']

[['извещение', 'проведение', 'электронный', 'аукцион', 'закупка'],
 ['общий', 'информация'],
 ['номер', 'извещение'],
 ['наименование', 'объект', 'закупка'],
 ['поставка', 'бумага', 'офисный', 'техника'],
 ['способ', 'определение', 'поставщик', 'подрядчик', 'исполнитель'],
 ['электронный', 'аукцион'],
 ['наименование', 'электронный'],
 ['объект', 'закупка', 'российский', 'рубль'],
 ['тип', 'объект', 'закупка'],
 ['наименование', 'товар', 'работа', 'услуга', 'окпд', 'ктереть'],
 ['код', 'позиция'],
 ['характеристика', 'товар', 'работа', 'услуга'],
 ['единица', 'измерение'],
 ['количество', 'объём', 'работа', 'услуга'],
 ['цена', 'единица', 'измерение'],
 ['стоимость', 'позиция'],
 ['наименование', 'характеристика'],
 ['значение', 'характеристика'],
 ['единица', 'измерение', 'характеристика'],
 ['бумага', 'офисный', 'техника', 'белый'],
 ['пачка'],
 ['количество', 'лист', 'пачка'],
 ['масса', 'бумага', 'площадь', 'м', '≥'],
 ['формат'],
 ['итого', 'российский', 'рубль'],
 ['преимущество'

In [ ]:
contract_enforcement_lemmatized = []
for i in range(len(contract_enforcement)):
  sentence = contract_enforcement[i]
  sentence = lemmatization(sentence, stopwords_ru, morph)

  if sentence:
    contract_enforcement_lemmatized.append(sentence)

In [ ]:
garantee_enforcement_lemmatized = []
for i in range(len(garantee_enforcement)):
  sentence = garantee_enforcement[i]
  sentence = lemmatization(sentence, stopwords_ru, morph)
  if sentence: 
    garantee_enforcement_lemmatized.append(sentence)

In [ ]:
nonetype_lemmatized = []
for i in range(len(nonetype_sentences)):
  sentence = nonetype_sentences[i]
  sentence = lemmatization(sentence, stopwords_ru, morph)

  if sentence and len(sentence) > 1: 
    nonetype_lemmatized.append(sentence)

In [ ]:
len(contract_enforcement_lemmatized), len(garantee_enforcement_lemmatized), len(nonetype_lemmatized)

In [ ]:
with open('contract_enforcement_lemmatized.pkl', 'wb') as fp:
  pickle.dump(contract_enforcement_lemmatized, fp)

In [ ]:
with open('garantee_enforcement_lemmatized.pkl', 'wb') as fp:
  pickle.dump(garantee_enforcement_lemmatized, fp)

In [ ]:
with open('nonetype_lemmatized.pkl', 'wb') as fp:
  pickle.dump(nonetype_lemmatized, fp)

In [ ]:
df_splitted.to_json('train_lemmatized.json')

### Просмотр ошибок разделения на предложения

In [ ]:
not_found.head()

,id,text,label,target,not_found
0,462513308,Приложение № 3 ГОСУДАРСТВЕННЫЙ КОНТРАКТ (ПРОЕК...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта устана...,0.0
1,988172508,Идентификационный код закупки: 222637800390863...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0
2,359099757,Идентификационный код закупки: 223631102474263...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0
3,967226407,Идентификационный код закупки: 222633500240263...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0
4,19349887,Идентификационный код закупки: 222634000559063...,обеспечение исполнения контракта,Размер обеспечение исполнения контракта устана...,0.0


In [ ]:
random_str = randint(0, len(not_found)-1)
not_found_split = splitter(not_found.loc[random_str, 'text'])
for i in not_found_split:
  print(i)
print('target:')
not_found.loc[random_str, 'target'] 

УТВЕРЖДАЮ
Генеральный директор АО «АТА» ________ А.В.
Балебанов «___» ________ 2022 года ЗАКУПОЧНАЯ ДОКУМЕНТАЦИЯ
Открытый одноэтапный конкурс в электронной форме без предварительного квалификационного отбора на поставку автомобильного топлива по пластиковым картам для АО «АТА» г.
Москва том 1 «общая и коммерческая части» СОГЛАСОВАНО:
Начальник отдела закупок _________________ С.Г.
Гаврилов 2021
Нагатинская улица, д.4а, Москва, 115230
Тел.: (495) 645-82-77, (495) 645-82-66
Факс: (499) 611-30-20,
E-mail: info@oaoata.ru ОГРН 1057749068002, ИНН/КПП 7724560930/772401001 СОДЕРЖАНИЕ ЗАКУПОЧНАЯ ДОКУМЕНТАЦИЯ 1 СОДЕРЖАНИЕ
1. ИЗВЕЩЕНИЕ
О ПРОВЕДЕНИИ ЗАКУПКИ 3 ЧАСТЬ 1
2. ТРЕБОВАНИЯ.
ДОКУМЕНТЫ. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ
В ЗАКУПКЕ.
2.1. ТРЕБОВАНИЯ.
ДОКУМЕНТЫ, ПОДТВЕРЖДАЮЩИЕ СООТВЕТСТВИЕ УСТАНОВЛЕННЫМ ТРЕБОВАНИЯМ.
2.1.1.
Требования к участникам закупки
2.1.2.
Требования к продукции
2.1.3.
Требования к банкам-гарантам, предоставляющим обеспечение заявки
1.1. СОСТАВ ЗАЯВКИ НА УЧАСТИЕ
В ЗА ЧАСТЬ 3 Проект 